In [ ]:
!pip install ml_metrics
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from sklearn.model_selection import train_test_split
import heapq 
import ml_metrics
import random
from sklearn.metrics.pairwise import cosine_similarity
import scipy.sparse as sp
random.seed(415)

In [ ]:
# read in test results
test_results = pd.read_csv('test_results_with_personalized_reranking_results_added.csv')
test_results = pd.DataFrame(test_results, columns=['userId', 'actual', 'cf_predictions', 'cf_predictions_10','personalized_reranking_predictions'])

In [ ]:
# read in training data and testing data 
train_data = pd.read_csv('data_train_set.csv')
test_data = pd.read_csv('data_test_set.csv')

# combine training and testing data 
train_test_data = [train_data, test_data]
ratings = pd.concat(train_test_data)
ratings.reset_index(drop=True, inplace=True)

In [ ]:
# get first n actual predictions for user
def get_actual_predictions(user_id, test_results, n):
  recommendations= list(map(int, test_results.loc[test_results['userId'] == user_id]['actual'].item()[1:-1].split(", ")))
  recommendations = recommendations[0:n]
  return recommendations

# get first n recommendations for user
def get_users_predictions(user_id, test_results, n):
  recommendations =  list(map(int, test_results.loc[test_results['userId'] == user_id]['cf_predictions'].item()[1:-1].split(", ")))
  recommendations = recommendations[0:n]
  return recommendations

# get first n Personalized Reranking predictions for user
def get_pr_predictions(user_id, test_results, n):
  recommendations= list(map(int, test_results.loc[test_results['userId'] == user]['personalized_reranking_predictions'].item()[1:-1].split(", ")))
  return recommendations


user_id = test_results.userId.unique().tolist()

# change baseline predictions from string format into a list
cf_predictions_10 =[]
for user in user_id:
    cf_pred_10 = get_users_predictions(user, test_results, 10)
    cf_predictions_10.append(cf_pred_10)

test_results['cf_predictions_10'] = cf_predictions_10

# change actual predictions from string format into a list
actual_predictions = []
for user in user_id:
    actual_pred = get_actual_predictions(user, test_results, 10)
    actual_predictions.append(actual_pred)

test_results['actual'] = actual_predictions

# change Personalized Reranking predictions from string format into a list
pr_predictions=[]
for user in user_id:
    pr_pred = get_pr_predictions(user, test_results, 10)
    pr_predictions.append(pr_pred)

test_results['personalized_reranking_predictions'] = pr_predictions

test_results=test_results.set_index("userId")

In [ ]:
# separate movies into short head and long tail data frames
df_movies_count_ratings = pd.DataFrame(ratings.groupby('movieId').size(), columns=['count']).reset_index()
total_num = len(df_movies_count_ratings)
short_head = int(0.2 * total_num)
df_short_head = df_movies_count_ratings.nlargest(short_head, 'count')['movieId'].unique()
df_long_tail = df_movies_count_ratings.nsmallest(total_num-short_head, 'count')['movieId'].unique()

In [ ]:
# create movie tag relevance matrix
tagId_filename = 'genome-scores.csv'
tags_filename = 'genome-tags.csv'

unique_movies = ratings.movieId.unique()
df_tag_id = pd.read_csv(tagId_filename, usecols=['movieId', 'tagId', 'relevance'],dtype={'movieId': 'int32','tagId': 'int32', 'relevance': 'float32'})
df_tag_id = df_tag_id.loc[df_tag_id['movieId'].isin(unique_movies)]
df_tags = pd.read_csv(tags_filename, usecols=['tagId', 'tag'],dtype={'tagId': 'int32', 'tag': 'str'})
unique_tag_id = df_tag_id['tagId'].unique().tolist()
df_movie_tag_relevance = df_tag_id.pivot(index='movieId', columns='tagId', values='relevance')

In [ ]:
df_tag_id.tagId.unique()

In [ ]:
# random walk helper function, performs random walk with restart using restart
# probability beta until convergence
def random_walk(movieId, model, beta):
  length = len(model)
  r = np.zeros(length)
  location = np.where(model.columns == movieId)
  r[location] = 1
  start = r.copy()
  for i in range(10):
    r_new = (1-beta)*model@r + (beta)*start
    if (np.allclose(r_new, r)):
      break
    else:
      r=r_new
  r /= r.sum()
  return r

In [ ]:
# read in user preference csv
df_user_prefs = pd.read_csv("user_preferences.csv")
df_user_prefs=df_user_prefs.set_index("userId")
user_mean = df_user_prefs['short_head_prefs'].mean()

In [ ]:
# random walk with restart with weighted edges (by inner product of two movies and their tags)
def rwr_reselection(user, df_tag_id, df_movies_count_ratings, test, df_short_head, df_long_tail, df_movie_tag_relevance, df_user_prefs, user_mean, beta):
  # get user recommendations using the user id
  user_pref = df_user_prefs.loc[user]['short_head_prefs'].item()
  starting_recommendations = list(map(int, test.loc[user]['cf_predictions'][1:-1].split(", ")))[0:beta]
  starting_recommendations = [x for x in starting_recommendations if x in df_movie_tag_relevance.index]

  # create transition matrix of probabilities
  df_probability = df_movie_tag_relevance.loc[starting_recommendations][:]
  df_transpose = df_probability.T
  df_probability = df_probability.dot(df_transpose)

  # zero diagonal values
  np.fill_diagonal(df_probability.values,0)

  # normalize all rows so that they sum to 1
  sum = df_probability.sum(axis=1).values
  df_probability = df_probability.div(sum, axis = 0)

  starting_recommendations_first_10 = test.loc[user]['cf_predictions_10']
  recommendations = []
  new_recs = []
  i = 0
  
  # find all recommendations in first 10 that needs to be reselected
  for m in starting_recommendations_first_10:
    if (m in df_short_head):
      new_recs.append(0)
      recommendations.append(i);
    else:
      new_recs.append(m)
    i= i + 1;

  # for each recommendation that needs to be reselecteed, perform RWR
  for index in recommendations:
    target_movie = starting_recommendations_first_10[index]
    if (target_movie in df_probability.columns and target_movie in df_short_head):
      weights = random_walk(target_movie, df_probability, user_pref)
      candidates = df_probability.columns
      replacement = np.random.choice(candidates, p=weights, size=1, replace=False)[0]
      while (replacement in new_recs):
        replacement = np.random.choice(candidates, p=weights, size=1, replace=False)[0]
      new_recs[index] = replacement
    else:
      new_recs[index] = target_movie
  return new_recs

In [ ]:
# make recommendations using random walk with restart reselection
rwr_recs = []
count = 0
beta = 50
for user in user_id:
  recs = rwr_reselection(user, df_tag_id, df_movies_count_ratings, test_results, df_short_head, df_long_tail, df_movie_tag_relevance, df_user_prefs, user_mean, beta)
  rwr_recs.append(recs)
  count = count + 1
  print(count, ": ", user)

test_results['rwr_predictions'] = rwr_recs
actual = list(test_results.actual.values)

In [ ]:
test_results['rwr_predictions'] = rwr_recs
actual = list(test_results.actual.values)

In [ ]:
# helper function to calculate prediction coverage
def prediction_coverage(predictions, all_items):
  predictions = set([j for i in predictions for j in i])
  prediction_coverage = len(predictions)/(len(all_items)*1.0)
  return prediction_coverage

# calculat prediction coverage
all_movies = ratings.movieId.unique().tolist()
rwr_coverage = prediction_coverage(rwr_recs, all_movies)
pr_coverage = prediction_coverage(pr_predictions, all_movies)
cf_coverage = prediction_coverage(cf_predictions_10, all_movies)
print('rwr: ', rwr_coverage)
print('pr: ', pr_coverage)
print('cf: ', cf_coverage)

In [ ]:
# calculate long tail coverage
def long_tail_coverage(predictions, df_long_tail) :
  predictions = set([j for i in predictions for j in i if j in df_long_tail])
  prediction_coverage = len(predictions)/(len(df_long_tail)*1.0)
  return prediction_coverage


rwr_coverage_long_tail = long_tail_coverage(rwr_recs, df_long_tail)
pr_coverage_long_tail = long_tail_coverage(pr_predictions, df_long_tail)
cf_coverage_long_tail = long_tail_coverage(cf_predictions_10, df_long_tail)
print('rwr: ', rwr_coverage_long_tail)
print('pr: ', pr_coverage_long_tail)
print('cf: ', cf_coverage_long_tail)

In [ ]:
# helper function to calculate novelty
def novelty(predictions, movie_counts, num_users, len_list):
  self_info = []
  for l in predictions:
    info = 0
    for i in l:
      info += np.sum(-np.log2(movie_counts[i]/num_users))
    self_info.append(info/len_list)
  return sum(self_info)/len(predictions)


movie_counts = dict(ratings.movieId.value_counts())
users = ratings.userId.unique().tolist()
rwr_novelty = novelty(rwr_recs, movie_counts, len(user_id), 10)
cf_novelty = novelty(cf_predictions_10, movie_counts, len(user_id), 10)
pr_novelty = novelty(pr_predictions, movie_counts, len(user_id), 10)
print('rwr: ', rwr_novelty)
print('pr: ', pr_novelty)
print('cf: ', cf_novelty)

In [ ]:
# calculate MAP@K
rwr_mapk = []
for K in np.arange(1, 11):
  rwr_mapk.extend([ml_metrics.mapk(actual, rwr_recs, k=K)])
print('rwr: ', rwr_mapk)

pr_mapk = []
for K in np.arange(1, 11):
    pr_mapk.extend([ml_metrics.mapk(actual, pr_predictions, k=K)])
print('pr: ', pr_mapk)

cf_mapk = []
for K in np.arange(1, 11):
    cf_mapk.extend([ml_metrics.mapk(actual, cf_predictions_10, k=K)])
print('cf: ', cf_mapk)

In [ ]:
# helper function to calculate personalization
def personalization(predictions):
  # create sparse matrix
  sparse_matrix = pd.DataFrame(data=predictions).reset_index().melt(id_vars='index', value_name='movies')
  sparse_matrix = pd.notna(sparse_matrix[['index', 'movies']].pivot(index='index', columns='movies', values='movies'))
  sparse_matrix = sp.csr_matrix(sparse_matrix.values)

  # calculate cosine similarity of upper triangle
  similarity = cosine_similarity(X=sparse_matrix, dense_output=False)
  upper_triangle = np.triu_indices(similarity.shape[0], k=1)
  return 1 - np.mean(similarity[upper_triangle])

rwr_personalization = personalization(rwr_recs)
pr_personalization = personalization(pr_predictions)
cf_personalization = personalization(cf_predictions_10)
print('rwr: ', rwr_personalization)
print('pr: ', pr_personalization)
print('cf: ', cf_personalization)

In [ ]:
# organize movie features
rated_movies = ratings.movieId.tolist()
movies = pd.read_csv('movies.csv')
movies = movies.query('movieId in @rated_movies').set_index("movieId", inplace=False, drop=True)
movies = movies.genres.str.split("|", expand=True).reset_index(inplace=False)
movies = pd.melt(movies, id_vars='movieId', value_vars=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
movies = movies.drop_duplicates("movieId", inplace=False).set_index('movieId', inplace=False)
movies = pd.get_dummies(movies.value)

In [ ]:
# helper function to calculate intra-list similarity
def ils(predictions, movies):
  ils = []
  for u in range(len(predictions)):
    # create sparse matrix
    sparse_matrix = movies.loc[predictions[u]]
    sparse_matrix = sp.csr_matrix(sparse_matrix.values)
    
    # calculate cosine similarity
    similarity = cosine_similarity(X=sparse_matrix, dense_output=False)
    upper_triangle = np.triu_indices(similarity.shape[0], k=1)
    ils.append(np.mean(similarity[upper_triangle]))
  return np.mean(ils)

rwr_similarity = ils(rwr_recs, movies)
pr_similarity = ils(pr_predictions, movies)
cf_similarity = ils(cf_predictions_10, movies)
print('rwr: ', rwr_similarity)
print('pr: ', pr_similarity)
print('cf: ', cf_similarity)

In [ ]:
# helper function to calculate movie category proportion
def percent_in_rec(userId, result, movie_cat, test_results):
  recs = test_results.loc[userId][result]
  recs_long_tail = [x for x in recs if x in movie_cat]
  return len(recs_long_tail)/len(recs)

# calculate average short-head proportion in recommendation lists
avg_short_head = 0;
for user in user_id:
  avg_short_head += percent_in_rec(user, 'rwr_predictions', df_short_head, test_results)
print('rwr: ', avg_short_head/len(user_id))

avg_short_head = 0;
for user in user_id:
  avg_short_head += percent_in_rec(user, 'personalized_reranking_predictions', df_short_head, test_results)
print('pr: ', avg_short_head/len(user_id))

avg_short_head = 0;
for user in user_id:
  avg_short_head += percent_in_rec(user, 'cf_predictions_10', df_short_head, test_results)
print('cf: ', avg_short_head/len(user_id))

In [ ]:
# calculate average long-tail proportion in recommendation lists
avg_long_tail = 0;
for user in user_id:
  avg_long_tail += percent_in_rec(user, 'rwr_predictions', df_long_tail, test_results)
print('rwr: ', avg_long_tail/len(user_id))

avg_long_tail = 0;
for user in user_id:
  avg_long_tail += percent_in_rec(user, 'personalized_reranking_predictions', df_long_tail, test_results)
print('pr: ', avg_long_tail/len(user_id))

avg_long_tail = 0;
for user in user_id:
  avg_long_tail += percent_in_rec(user, 'cf_predictions_10', df_long_tail, test_results)
print('cf: ', avg_long_tail/len(user_id))